In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os


In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.0001


In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
real_path = "/content/drive/MyDrive/dataset/real"
fake_path = "/content/drive/MyDrive/dataset/fake"


In [ ]:
dataset_path = "/content/drive/MyDrive/dataset"
dataset = ImageFolder(root=dataset_path, transform=transform)


In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")


Epoch [1/10], Loss: 0.1578, Accuracy: 0.9300
Epoch [2/10], Loss: 0.0590, Accuracy: 0.9300
Epoch [3/10], Loss: 0.0513, Accuracy: 0.9300
Epoch [4/10], Loss: 0.0499, Accuracy: 0.9300
Epoch [5/10], Loss: 0.0500, Accuracy: 0.9300
Epoch [6/10], Loss: 0.0490, Accuracy: 0.9300
Epoch [7/10], Loss: 0.0492, Accuracy: 0.9300
Epoch [8/10], Loss: 0.0482, Accuracy: 0.9856
Epoch [9/10], Loss: 0.0479, Accuracy: 0.9991
Epoch [10/10], Loss: 0.0475, Accuracy: 0.9991


In [ ]:
model_save_path = "/content/drive/MyDrive/deepfake_detector.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model training complete and saved as {model_save_path}")


Model training complete and saved as /content/drive/MyDrive/deepfake_detector.pth


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import cv2
import numpy as np
from PIL import Image
import os


In [ ]:
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (128 // 8) * (128 // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/deepfake_detector.pth", map_location=device))
model.eval()


DeepFakeDetector(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
def predict_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_predictions = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 10 != 0:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            prediction = "Real" if output.item() > 0.5 else "Fake"

        frame_predictions.append(prediction)

    cap.release()
    real_count = frame_predictions.count("Real")
    fake_count = frame_predictions.count("Fake")

    final_prediction = "Fake" if fake_count > real_count else "Real"

    print(f"Final Prediction: {final_prediction}")
    print(f"Real Frames: {real_count}, Fake Frames: {fake_count}")



In [ ]:
video_path = "/content/drive/MyDrive/Test/videoplayback.mp4"
predict_video(video_path)


Final Prediction: Real
Real Frames: 234, Fake Frames: 0


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model
model_path = "/content/drive/MyDrive/deepfake_detector.h5"
model = load_model(model_path)
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image
def predict_image(image_path):
    image_tensor = preprocess_image(image_path)
    prediction = model.predict(image_tensor)
    return "Fake" if prediction[0][0] > 0.5 else "Real"

image_path = "/content/drive/MyDrive/dataset/real/0.jpg"
result = predict_image(image_path)
print(f"The image is predicted as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
The image is predicted as: Real
